https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/

In [ ]:
# prepare data for lstm
import matplotlib.pyplot as plt
import numpy as np
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [ ]:
# https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df =  DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(-i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
# load dataset
dataset = read_csv('SeoulBikeData_168.csv', encoding= 'unicode_escape', header=0, index_col=0)

values = dataset.values

In [ ]:
# encode the categorical variables into integers
label_encoder = LabelEncoder()
values[:,12] = label_encoder.fit_transform(values[:,12])
values[:,10] = label_encoder.fit_transform(values[:,10])
values[:,11] = label_encoder.fit_transform(values[:,11])

In [ ]:
# ensure all data is float
values = values.astype('float32')
# normalize features
#scaler = MinMaxScaler(feature_range=(0, 1))
#scaled = scaler.fit_transform(values)

In [ ]:
# frame as supervised learning
hours_to_consider = 168
reframed = series_to_supervised(values, hours_to_consider, 1)

# drop columns we don't want to predict
columns_to_keep = list()
for i in range(337,reframed.shape[1]+1,14):
  columns_to_keep.append(i)

n_cols = [i for i in range (337,reframed.shape[1]+1)]
for i in columns_to_keep:
    n_cols.remove(i)
n_cols2 = [x - 1 for x in n_cols]
print(reframed)
reframed = reframed.drop(reframed.columns[[n_cols2]], axis=1, inplace=False)
print(reframed)

      var1(t-168)  var2(t-168)  var3(t-168)  ...  var12(t)  var13(t)  var14(t)
0           254.0          0.0         -5.2  ...       1.0       1.0      12.0
1           204.0          1.0         -5.5  ...       1.0       1.0      12.0
2           173.0          2.0         -6.0  ...       1.0       1.0      12.0
3           107.0          3.0         -6.2  ...       1.0       1.0      12.0
4            78.0          4.0         -6.0  ...       1.0       1.0      12.0
...           ...          ...          ...  ...       ...       ...       ...
8755       1003.0         19.0          4.2  ...       1.0       1.0      11.0
8756        764.0         20.0          3.4  ...       1.0       1.0      11.0
8757        694.0         21.0          2.6  ...       1.0       1.0      11.0
8758        712.0         22.0          2.1  ...       1.0       1.0      11.0
8759        584.0         23.0          1.9  ...       1.0       1.0      11.0

[8760 rows x 2366 columns]
      var1(t-168)  var2(

/usr/local/lib/python3.7/dist-packages/pandas/core/indexes/base.py:4114: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)


In [ ]:
import pandas as pd
my_list = pd.DataFrame(reframed)
my_list = my_list.columns.values.tolist()

In [ ]:
# split into train and test sets
values = reframed.values
train_ratio = 0.85
n_train_hours = int(365 * hours_to_consider * train_ratio)

train = values[:n_train_hours, :]
test = values[n_train_hours:, :]

# split into input and outputs
train_X, train_y = train[:, 1:], train[:, 0]
test_X, test_y = test[:, 1:], test[:, 0]

In [ ]:
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(8760, 1, 480) (8760,) (0, 1, 480) (0,)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.optimizers import SGD, Adam, RMSprop

from keras.metrics import MeanSquaredError
from keras.metrics import RootMeanSquaredError
from keras.metrics import MeanAbsoluteError
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

!pip install keras-tuner
from kerastuner.tuners import RandomSearch, Hyperband
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import Reshape

def adj_r2(r2):
  n = train_X.shape[0]
  p = train_X.shape[2]
  return (1-(1-r2)*(n-1)/(n-p-1))

     |████████████████████████████████| 71kB 4.5MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=e3ce3154efa3f1685fab6b639f803fdad7657caf364024517800c1cc68ed95f1
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=41a25ab487a670aab6f17ea9369024232af1f70ecacaab143c3910f62c38bf50
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [11]:
def model_builder(hp):
  model = Sequential()

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units1', min_value=32, max_value=512, step=32)
  model.add(LSTM(hp_units,
                 input_shape=(train_X.shape[1], train_X.shape[2]),
                 return_sequences=True
                 ))
  model.add(BatchNormalization(axis=-1, 
                            momentum=0.99,
                            epsilon=0.001,
                            center=True,
                            scale=True,
                            beta_initializer='zeros',
                            gamma_initializer='ones',
                            moving_mean_initializer='zeros',
                            moving_variance_initializer='ones',
                            beta_regularizer=None,
                            gamma_regularizer=None,
                            beta_constraint=None,
                            gamma_constraint=None
                            ))

  hp_units2 = hp.Int('units2', min_value=32, max_value=512, step=32)
  model.add(LSTM(hp_units2, input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences=True))
  model.add(BatchNormalization(axis=-1, 
                            momentum=0.99,
                            epsilon=0.001,
                            center=True,
                            scale=True,
                            beta_initializer='zeros',
                            gamma_initializer='ones',
                            moving_mean_initializer='zeros',
                            moving_variance_initializer='ones',
                            beta_regularizer=None,
                            gamma_regularizer=None,
                            beta_constraint=None,
                            gamma_constraint=None))
  
  hp_units3 = hp.Int('units3', min_value=32, max_value=512, step=32)
  model.add(LSTM(hp_units3, input_shape=(train_X.shape[1], train_X.shape[2])))
  model.add(BatchNormalization(axis=-1, 
                            momentum=0.99,
                            epsilon=0.001,
                            center=True,
                            scale=True,
                            beta_initializer='zeros',
                            gamma_initializer='ones',
                            moving_mean_initializer='zeros',
                            moving_variance_initializer='ones',
                            beta_regularizer=None,
                            gamma_regularizer=None,
                            beta_constraint=None,
                            gamma_constraint=None))
  model.add(Dense(1))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('lr', values=[1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6])
  
  model.compile(optimizer=Adam(hp_learning_rate),
                loss="mse",
                metrics=['accuracy', MeanSquaredError(), RootMeanSquaredError(),MeanAbsoluteError()])

  return model


tuner = Hyperband(model_builder,
                  objective='mean_squared_error',
                  max_epochs=100,
                  factor=10,
                  directory='my_dir',
                  hyperband_iterations=5,
                  project_name='logss')

stop_early = EarlyStopping(monitor='val_loss', patience=100)

tuner.search(train_X, train_y, epochs=1000, validation_split=0.15, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
Layer 1: {best_hps.get('units1')}
Layer 2: {best_hps.get('units2')}
Layer 3: {best_hps.get('units3')}

Learning rate: {best_hps.get('lr')}.""")

Trial 655 Complete [00h 09m 54s]
mean_squared_error: 96318.953125

Best mean_squared_error So Far: 86678.3359375
Total elapsed time: 11h 38m 03s
INFO:tensorflow:Oracle triggered exit

Layer 1: 128
Layer 2: 480
Layer 3: 416

Learning rate: 0.0001.


In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(train_X, train_y, epochs=1000, validation_split=0.15)

Epoch 1/1000
233/233 [==============================] - 15s 44ms/step - loss: 913675.1330 - accuracy: 0.0117 - mean_squared_error: 913675.1330 - root_mean_squared_error: 955.3606 - mean_absolute_error: 697.5239 - val_loss: 940736.3750 - val_accuracy: 0.0624 - val_mean_squared_error: 940736.3750 - val_root_mean_squared_error: 969.9156 - val_mean_absolute_error: 787.5779
Epoch 2/1000
233/233 [==============================] - 9s 38ms/step - loss: 821599.9482 - accuracy: 0.0201 - mean_squared_error: 821599.9482 - root_mean_squared_error: 906.1421 - mean_absolute_error: 666.1917 - val_loss: 917517.4375 - val_accuracy: 0.0616 - val_mean_squared_error: 917517.4375 - val_root_mean_squared_error: 957.8713 - val_mean_absolute_error: 784.2990
Epoch 3/1000
233/233 [==============================] - 9s 38ms/step - loss: 830334.1031 - accuracy: 0.0215 - mean_squared_error: 830334.1031 - root_mean_squared_error: 911.1098 - mean_absolute_error: 674.9544 - val_loss: 902143.4375 - val_accuracy: 0.0662 

In [ ]:
root_mean_squared_error = history.history['val_root_mean_squared_error']
best_epoch = root_mean_squared_error.index(min(root_mean_squared_error)) + 1
print('Best epoch: %d' % (best_epoch,))

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(train_X, train_y, epochs=best_epoch, validation_split=0.15)

In [ ]:
# eval_result = hypermodel.evaluate(img_test, label_test)
# print("[test loss, test accuracy]:", eval_result)

In [ ]:
# # design network
# model = Sequential()
# model.add(LSTM(224, input_shape=(train_X.shape[1], train_X.shape[2])))
# model.add(BatchNormalization(axis=-1, 
#                             momentum=0.99,
#                             epsilon=0.001,
#                             center=True,
#                             scale=True,
#                             beta_initializer='zeros',
#                             gamma_initializer='ones',
#                             moving_mean_initializer='zeros',
#                             moving_variance_initializer='ones',
#                             beta_regularizer=None,
#                             gamma_regularizer=None,
#                             beta_constraint=None,
#                             gamma_constraint=None
#                             ))
# #model.add(Dropout(rate=0.25))
# model.add(Dense(1))


# model.compile(loss='mae',
#               optimizer=SGD(lr=1e-05),
#               metrics=["accuracy", 
#                        MeanSquaredError(), 
#                        RootMeanSquaredError(),
#                        MeanAbsoluteError()])

# # fit network
# history = model.fit(train_X, 
#                     train_y,
#                     validation_split=0.15,
#                     epochs=166,
#                     batch_size=60,
#                     verbose=2,
#                     shuffle=False)

# # plot history
# pyplot.plot(history.history['loss'], label='train')
# pyplot.plot(history.history['val_loss'], label='test')
# pyplot.legend()
# pyplot.show()

# # make a prediction
# yhat = model.predict(test_X)
# test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))

# # invert scaling for forecast
# oi = np.random.randint(1,101,size=(853,12))
# inv_yhat = concatenate((yhat, oi), axis=1)
# print(inv_yhat.shape)
# inv_yhat = scaler.inverse_transform(inv_yhat)
# inv_yhat = inv_yhat[:,0]

# # invert scaling for actual
# test_y = test_y.reshape((len(test_y), 1))
# inv_y = concatenate((test_y, oi), axis=1)
# inv_y = scaler.inverse_transform(inv_y)
# inv_y = inv_y[:,0]

# # calculate R2
# r2 = r2_score(inv_y, inv_yhat)
# adj_r2 = adj_r2(r2)
# print('Test R2: %.3f' % r2)
# print('Test ADJ R2: %.3f' % adj_r2)

In [ ]:
# summarize history for accuracy
plt.plot(history.history['root_mean_squared_error'])
plt.plot(history.history['val_root_mean_squared_error'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()